In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import ssl

In [2]:
key_excel = pd.read_excel('Keywords.xlsx')

In [3]:
Kywrd_1 = key_excel['Unnamed: 1'].to_list()[3:-9]
Kywrd_1 = [item.replace('国際潮流①：\n', '') for item in Kywrd_1]
Kywrd_1 = [item.replace('国際潮流②：\n', '') for item in Kywrd_1]
Kywrd_1 = [item.replace('国際潮流③：\n', '') for item in Kywrd_1]
Kywrd_2_lst = ['Amazon ','Google ','Facebook ','Apple ','Microsoft ','Salesforce ']
Keyword = []
for gafamS in Kywrd_2_lst:
    Keyword.append([gafamS + item for item in Kywrd_1])
def flatten(nested_list):
    """2重のリストをフラットにする関数"""
    return [e for inner_list in nested_list for e in inner_list]

Keyword = flatten(Keyword)

In [19]:

df_lst = []
kywrd_lst = []
# 70キーワード×30検索リストが限界
for search_keyword in Keyword:
    print('【検索した単語】{}'.format(search_keyword))
    #検索順位取得処理
    #Google検索の実施
    search_url = 'https://www.google.co.jp/search?hl=ja&num=30&q=' + search_keyword
    res_google = requests.get(search_url)
    #Responseオブジェクトが持つステータスコードが200番台(成功)以外だったら、エラーメッセージを吐き出してスクリプトを停止します。
    res_google.raise_for_status()
    print("Google検索結果を取得")

    #res_google.textは、検索結果のページのHTML
    bs4_google = BeautifulSoup(res_google.text, 'lxml')
    google_search_page = bs4_google.select('div.kCrYT>a')

    #rank:検索順位
    rank = 1
    site_rank = []
    site_title = []
    site_url = []

    for site in google_search_page:
        try:    
            site_title.append(site.select('h3.zBAuLc')[0].text)
            site_url.append(site.get('href').split('&sa=U&')[0].replace('/url?q=', ''))
            site_rank.append(rank)
            rank +=1
        except IndexError:
            continue

    print("以上")

    df = pd.DataFrame({'順位':site_rank, '検索ワード':search_keyword,  'タイトル':site_title, 'URL':site_url})
    df_lst.append(df)

In [20]:
a = pd.concat(df_lst)

In [21]:
a.to_csv('検索結果.csv',index=None, encoding='utf_8_sig')